In [1]:
import pandas as pd
import win32com.client

In [2]:

instCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
print(instCpCybos.IsConnect)

1


# 주식 종목 확인하기
+ GetData의 첫번째 input이 0이면 종목코드, 1:종목명, 2:Fullcode

In [3]:
instCpStockCode = win32com.client.Dispatch("CpUtil.CpStockCode")
print(instCpStockCode.GetData(0, 0))
print(instCpStockCode.GetData(1, 0))
print(instCpStockCode.GetData(2, 0))

A000020
동화약품
KR7000020008


# 특정 회사의 종목 코드와 인덱스 번호 확인하기

In [4]:
naverCode = instCpStockCode.NameToCode('NAVER')
naverIndex = instCpStockCode.CodeToIndex(naverCode)
print(naverCode)
print(naverIndex)

A035420
870


In [5]:
print(instCpStockCode.GetData(0, 870))
print(instCpStockCode.GetData(1, 870))
print(instCpStockCode.GetData(2, 870))


A035420
NAVER
KR7035420009


# 종목 코드 리스트 확인하는 법

In [6]:
instCpCodeMgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")
codeList = instCpCodeMgr.GetStockListByMarket(1)
# print(codeList)

In [7]:
# 종목 코드 딕셔너리로 저장
kospi = {}

for code in codeList:
    name = instCpCodeMgr.CodeToName(code)
    kospi[code] = name

kospi



{'A000020': '동화약품',
 'A000040': 'KR모터스',
 'A000050': '경방',
 'A000060': '메리츠화재',
 'A000070': '삼양홀딩스',
 'A000075': '삼양홀딩스우',
 'A000080': '하이트진로',
 'A000087': '하이트진로2우B',
 'A000100': '유한양행',
 'A000105': '유한양행우',
 'A000120': 'CJ대한통운',
 'A000140': '하이트진로홀딩스',
 'A000145': '하이트진로홀딩스우',
 'A000150': '두산',
 'A000155': '두산우',
 'A000157': '두산2우B',
 'A000180': '성창기업지주',
 'A000210': 'DL',
 'A000215': 'DL우',
 'A000220': '유유제약',
 'A000225': '유유제약1우',
 'A000227': '유유제약2우B',
 'A000230': '일동홀딩스',
 'A000240': '한국앤컴퍼니',
 'A000270': '기아',
 'A000300': '대유플러스',
 'A000320': '노루홀딩스',
 'A000325': '노루홀딩스우',
 'A000370': '한화손해보험',
 'A000390': '삼화페인트',
 'A000400': '롯데손해보험',
 'A000430': '대원강업',
 'A000480': '조선내화',
 'A000490': '대동',
 'A000500': '가온전선',
 'A000520': '삼일제약',
 'A000540': '흥국화재',
 'A000545': '흥국화재우',
 'A000547': '흥국화재2우B',
 'A000590': 'CS홀딩스',
 'A000640': '동아쏘시오홀딩스',
 'A000650': '천일고속',
 'A000660': 'SK하이닉스',
 'A000670': '영풍',
 'A000680': 'LS네트웍스',
 'A000700': '유수홀딩스',
 'A000720': '현대건설',
 'A000725': '현대건설우

In [8]:
kospi_list = list(kospi.items())
kospi_list = pd.DataFrame(kospi_list, columns =['code', 'name'])
kospi_list.head()
# kospi_list

,code,name
0,A000020,동화약품
1,A000040,KR모터스
2,A000050,경방
3,A000060,메리츠화재
4,A000070,삼양홀딩스


In [32]:
# csv로 저장
kospi_list.to_csv("kospi_list.csv", encoding="euc-kr", index = None)

In [31]:
tmp = pd.read_csv("kospi_list.csv")
tmp.head()

,Unnamed: 0,code,name
0,0,A000020,동화약품
1,1,A000040,KR모터스
2,2,A000050,경방
3,3,A000060,메리츠화재
4,4,A000070,삼양홀딩스


# 주식 과거 정보 가져오기
+ StockChart class

In [34]:
instStockChart = win32com.client.Dispatch("CpSysDib.Stockchart")

# 0:종목코드 , 1:기간(ord('1')) or 개수(ord('2')), 4:요청개수, 5:데이터의 종류(여기서 5는 종가), 6:데이터 차트의 종류(D는 day), 9:수정주가 반영 여부
# 정보를 얻기위한 필요한 입력 조건
instStockChart.SetInputValue(0, "A035420") #네이버
instStockChart.SetInputValue(1, ord('2'))
instStockChart.SetInputValue(4, 10)
instStockChart.SetInputValue(5, 5)
instStockChart.SetInputValue(6, ord('D'))
instStockChart.SetInputValue(9, ord('1'))

instStockChart.BlockRequest() # 데이터 요청
numData = instStockChart.GetHeaderValue(3) # 수신된 데이터의 개수
print(numData)


10


In [35]:
# 네이버 종가 데이터 출력
for i in range(numData):
    print(instStockChart.GetDataValue(0,i))

388500
393500
373000
370500
382000
388000
387000
392500
403000
405500


In [36]:
# 날짜 기준으로 요청

instStockChart.SetInputValue(0, "A035420") #네이버
instStockChart.SetInputValue(1, ord('1')) # 기간으로 요청
instStockChart.SetInputValue(2, '20211008') # 종료기간
instStockChart.SetInputValue(3, '20210924') # 시작 기간
# instStockChart.SetInputValue(4, 10)
instStockChart.SetInputValue(5, (0,5, 6, 8))
instStockChart.SetInputValue(6, ord('D'))
instStockChart.SetInputValue(9, ord('1'))

instStockChart.BlockRequest() # 데이터 요청
numData = instStockChart.GetHeaderValue(3) # 수신된 데이터의 개수
numField = instStockChart.GetHeaderValue(1)
print(numData)

for i in range(numData):
    for j in range(numField):

        print(instStockChart.GetDataValue(j,i), end = " ")

    print("")

10
20211008 388500 -5000 542514 
20211007 393500 20500 832068 
20211006 373000 2500 826716 
20211005 370500 -11500 828794 
20211001 382000 -6000 402239 
20210930 388000 1000 611128 
20210929 387000 -5500 664216 
20210928 392500 -10500 857556 
20210927 403000 -2500 455667 
20210924 405500 5500 574341 
